### Görev 1: Parametre Verimliliğini Kanıtlama (The "Efficiency" Challenge)

Aşağıdaki kod bloğunda eksik bırakılan yerleri tamamlayın. Amacımız, LoRA uygulandığında eğitilebilir parametre sayısının ne kadar düştüğünü (veya ne kadar az parametre ile eğitim yapıldığını) matematiksel olarak görmektir.

In [1]:
# Görev 1: Parametre Verimliliği — GPT-2 + LoRA
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
import torch

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Eğitimde açılacak parametre sayısını yazdır
def print_trainable_parameters(m):
    total = 0
    trainable = 0
    for p in m.parameters():
        c = p.numel()
        total += c
        if p.requires_grad:
            trainable += c
    ratio = (trainable / total) * 100 if total > 0 else 0
    print(f"Toplam parametre: {total}")
    print(f"Eğitilebilir parametre: {trainable} ({ratio:.2f}%)")

print("Baz model (GPT-2) — parametre durumu:")
print_trainable_parameters(model)

# LoRA yapılandırması
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["c_attn", "c_proj"]
)

peft_model = get_peft_model(model, peft_config)
print("\nLoRA uygulanmış model — parametre durumu:")
print_trainable_parameters(peft_model)


c:\Users\ASUS\Documents\GitHub\HuaweiStdDev-UludagUni-AIClub\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ASUS\Documents\GitHub\HuaweiStdDev-UludagUni-AIClub\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as 

Baz model (GPT-2) — parametre durumu:
Toplam parametre: 124439808
Eğitilebilir parametre: 124439808 (100.00%)

LoRA uygulanmış model — parametre durumu:
Toplam parametre: 125250816
Eğitilebilir parametre: 811008 (0.65%)


c:\Users\ASUS\Documents\GitHub\HuaweiStdDev-UludagUni-AIClub\.venv\Lib\site-packages\peft\tuners\lora\layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


### Görev 2: LoRA Katmanlarını Gözlemleme (The "Architecture" Challenge)

LoRA, modelin mevcut katmanlarına (genellikle Attention bloklarına) küçük matrisler ekler. Aşağıdaki kodu çalıştırarak `lora_A` ve `lora_B` katmanlarının modelin neresine eklendiğini çıktı üzerinde tespit edin.

In [2]:
# Görev 2: LoRA Katmanlarını Gözlemleme
# Amaç: Modelde eklenen lora_A / lora_B katmanlarının yerlerini yazdırmak

found = []
for name, module in peft_model.named_modules():
    has_a = hasattr(module, 'lora_A')
    has_b = hasattr(module, 'lora_B')
    if has_a or has_b:
        found.append(name)
        print(name)

print(f"\nToplam LoRA adapte edilmiş alt modül sayısı: {len(found)}")


base_model.model.transformer.h.0.attn.c_attn
base_model.model.transformer.h.0.attn.c_proj
base_model.model.transformer.h.0.mlp.c_proj
base_model.model.transformer.h.1.attn.c_attn
base_model.model.transformer.h.1.attn.c_proj
base_model.model.transformer.h.1.mlp.c_proj
base_model.model.transformer.h.2.attn.c_attn
base_model.model.transformer.h.2.attn.c_proj
base_model.model.transformer.h.2.mlp.c_proj
base_model.model.transformer.h.3.attn.c_attn
base_model.model.transformer.h.3.attn.c_proj
base_model.model.transformer.h.3.mlp.c_proj
base_model.model.transformer.h.4.attn.c_attn
base_model.model.transformer.h.4.attn.c_proj
base_model.model.transformer.h.4.mlp.c_proj
base_model.model.transformer.h.5.attn.c_attn
base_model.model.transformer.h.5.attn.c_proj
base_model.model.transformer.h.5.mlp.c_proj
base_model.model.transformer.h.6.attn.c_attn
base_model.model.transformer.h.6.attn.c_proj
base_model.model.transformer.h.6.mlp.c_proj
base_model.model.transformer.h.7.attn.c_attn
base_model.model.